Визуализация комплексной функции:
$$f(z, \, \phi) = (z^2 - 2.5) (z - 2.5 i) (z + 2.5 i)
\frac {(z - 2 - i)^2} {(z - e^{i \phi})^2 (z - e^{2 i \phi})^2}.$$

Область визуализации:
$$\operatorname {Re} z \in [-3, 3], \quad \operatorname {Im} z \in [-3, 3], \quad \phi \in [0, 2 \pi).$$

In [1]:
QT_GUI, JS_LOSSLESS, MPEG_LOSSY, MPEG_LOSSLESS = range(4)
mode = MPEG_LOSSY

In [2]:
if mode == QT_GUI:
    %matplotlib qt
    
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Video
from matplotlib.colors import hsv_to_rgb

In [9]:
if not mode == QT_GUI:
    # %matplotlib inline
    # %config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
    plt.ioff()

# use typesetting program LaTeX 
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# use dark theme
# plt.style.use('dark_background')
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# set dpi
plt.rcParams['figure.dpi'] = 100

In [4]:
def z_to_rgb(z):
    # angle in (-pi, pi]
    # H in (0, 1]
    # H = (angle + pi) / 2pi = angle / 2pi + 1 / 2
    H = np.angle(z) / (2 * np.pi) + .5

    r = np.log2(1. + np.abs(z))
    S = (1. + np.abs(np.sin(2. * np.pi * r))) / 2.
    V = (1. + np.abs(np.cos(2. * np.pi * r))) / 2.

    rgb = hsv_to_rgb(np.dstack((H, S, V)))
    
    return rgb

In [11]:
f = lambda z, phi: (
    (z ** 2 - 2.5) * (z - 2.5 * 1j) * (z + 2.5 * 1j)
  * (z - 2 - 1j) ** 2 / ((z - np.exp(1j * phi)) ** 2
  * (z - np.exp(1j * 2 * phi)) ** 2)
)


r = 3
n = 1000
x = np.linspace(-r, r, n)
y = np.linspace(-r, r, n)
X, Y = np.meshgrid(x, y, sparse=True)
z = X + 1j * Y


fig = plt.figure()
ax = plt.subplot(xlabel='$\operatorname {Re} z$', ylabel='$\operatorname {Im} z$')
# ax = plt.subplot(xlabel='$\Re \, \{z\}$', ylabel='$\Im \, \{z\}$')

a = np.zeros(X.shape)
im = plt.imshow(a, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
ttl = ax.text(.5, 1.05, 'z', transform = ax.transAxes, va='center')


def init_func():
#     ttl.set_text('')
#     im.set_data(a)
    
    return im, ttl


# import time
# start = 0

def func(t):
#     global start

    data = z_to_rgb(f(z, phi=2 * np.pi * t))
    im.set_data(data)
    ttl.set_text(f"$f(z, \, \phi), \ \phi = 2 \pi \cdot {t:g}$")

#     end = time.time()
#     elapsed = end - start
#     start = end
    
#     print(f'Elapsed {elapsed * 1000} ms')#, end='\r')

    return im, ttl


total_time = 5  # sec
fps = 60
frames = round(total_time * fps)
t = np.linspace(0, 1, frames, endpoint=False)


ani = FuncAnimation(
    fig, func, init_func=init_func,
    frames=t, interval=1000 / fps,
#     blit=True
)


progress_callback = lambda i, n: print(f"Saving frame {i} of {n}", end=('\n' if i == n - 1 else '\r'))


# save frames
# ani.save('trip/trip.png', writer="imagemagick", progress_callback=progress_callback)


if mode == QT_GUI:
    plt.show()

if mode == JS_LOSSLESS:
    display(HTML(ani.to_jshtml()))
    plt.close()

if mode == MPEG_LOSSY:
    # display(HTML(ani.to_html5_video()))
    # ani.save('complex_f_lossy.mp4', codec='libx264')
    %time ani.save("complex_f_lossy.mp4", bitrate=10000, fps=fps, \
                   progress_callback=progress_callback)
    display(Video("complex_f_lossy.mp4", html_attributes="controls loop autoplay"))
    plt.close()

if mode == MPEG_LOSSLESS:
    %time ani.save("complex_f_lossless.mp4", fps=fps, codec='libx264', \
                   extra_args=['-preset', 'veryslow', '-crf', '0'], \
                   progress_callback=progress_callback)
    display(Video("complex_f_lossless.mp4", html_attributes="controls loop autoplay"))
    plt.close()

Saving frame 299 of 300
CPU times: user 2min 27s, sys: 23.3 s, total: 2min 50s
Wall time: 5min 12s
